In [12]:
#Step 1: Import Libraries
#!pip install openai
import openai
import os
import pandas as pd
#!pip install sqlalchemy
import sqlalchemy
#Import these libraries to setup a temp DB in RAM and PUSH Pandas DF to DB
from sqlalchemy import create_engine
from sqlalchemy import text

In [13]:
with open('api.txt', 'r') as f:
    openai.api_key = f.read().strip()

In [14]:
#Read in data
df = pd.read_csv("sales_data_sample.csv")

In [15]:
#Review data
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [16]:
#Create temp DB
temp_db = create_engine('sqlite:///memory:', echo = True)

In [21]:
#Push the DF to be in SQL DB
data = df.to_sql(name = "sales_table", con = temp_db)

2023-04-17 18:31:42,738 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-17 18:31:42,740 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales_table")
2023-04-17 18:31:42,741 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-17 18:31:42,743 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sales_table")
2023-04-17 18:31:42,745 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-17 18:31:42,747 INFO sqlalchemy.engine.Engine ROLLBACK
2023-04-17 18:31:42,753 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-17 18:31:42,756 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales_table (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	

In [22]:
#Connect to SQL DB
with temp_db.connect() as conn:
    results = conn.execute(text("SELECT SUM(SALES) FROM Sales"))

2023-04-17 18:32:05,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-17 18:32:05,261 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-04-17 18:32:05,262 INFO sqlalchemy.engine.Engine [generated in 0.00261s] ()
2023-04-17 18:32:05,264 INFO sqlalchemy.engine.Engine ROLLBACK


In [23]:
#Return Results
results.all()

[(10032628.85000001,)]

In [24]:
'''
We now need to create some handler functions for GPT-3 to understand
the table structure.

STEPS FOR THIS PROCESS:

1. Automatically create table structure definition from the pandas DF for 
OpenAI API

2. Grab user NLP input

3. Combine the prompts for an OpenAI API call

'''

'\nWe now need to create some handler functions for GPT-3 to understand\nthe table structure.\n\nSTEPS FOR THIS PROCESS:\n\n1. Automatically create table structure definition from the pandas DF for \nOpenAI API\n\n2. Grab user NLP input\n\n3. Combine the prompts for an OpenAI API call\n\n'

In [25]:
# SQLite SQL tables with thier properties:
# -----------------------------------------
# Employee (ID, Name, Department_ID)
# Department (ID, Name, Address)
# Salary_Payments (ID, Emplyee_ID, Amount, Date)
# -----------------------------------------
#Create a funnction for table definitions
def create_table_definition(df):
    prompt = """### sqlite SQL table, with its properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))
    
    return prompt

In [26]:
#Review results
print(create_table_definition(df))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [27]:
#Create function for input
def prompt_input():
    nlp_text = input("Enter desired information: ")
    return nlp_text

In [33]:
#Validate function
prompt_input()

Enter desired information: SELECT


'SELECT'

In [34]:
#Combine these functions into a single function
def combined(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"###A query to answer: {query_prompt}\nSELECT"
    return definition + query_init_string

In [35]:
#Grabbing natural language
nlp_text = prompt_input()

Enter desired information: 


In [36]:
#inserting table definition (DF + quert that does... + NLP)
prompt = combined(df, nlp_text)

In [37]:
print(prompt)

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    ###A query to answer: 
SELECT


In [39]:
response = openai.Completion.create(
            model = 'text-davinci-002',
            prompt = combined (df, nlp_text),
            temperature = 0,
            max_tokens = 150,
            top_p = 1.0,
            frequency_penalty = 0,
            presence_penalty = 0,
            stop = ["#", ";"]
)

In [40]:
response['choices'][0]['text']

" * FROM Sales WHERE STATUS = 'Shipped' AND YEAR_ID = 2003 AND QTR_ID = 3\n    "

In [41]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query = 'SELECT' + query
    return query

In [42]:
handle_response(response)

"SELECT * FROM Sales WHERE STATUS = 'Shipped' AND YEAR_ID = 2003 AND QTR_ID = 3\n    "

In [46]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-04-17 18:38:01,146 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-17 18:38:01,147 INFO sqlalchemy.engine.Engine SELECT * FROM Sales WHERE STATUS = 'Shipped' AND YEAR_ID = 2003 AND QTR_ID = 3
    
2023-04-17 18:38:01,148 INFO sqlalchemy.engine.Engine [generated in 0.00237s] ()
2023-04-17 18:38:01,149 INFO sqlalchemy.engine.Engine ROLLBACK


In [47]:
result.all()

[(2, 10134, 41, 94.74, 2, 3884.34, '7/1/2003 0:00', 'Shipped', 3, 7, 2003, 'Motorcycles', 95, 'S10_1678', 'Lyon Souveniers', '+33 1 46 62 7555', '27 rue du Colonel Pierre Avia', None, 'Paris', None, '75508', 'France', 'EMEA', 'Da Cunha', 'Daniel', 'Medium'),
 (3, 10145, 45, 83.26, 6, 3746.7, '8/25/2003 0:00', 'Shipped', 3, 8, 2003, 'Motorcycles', 95, 'S10_1678', 'Toys4GrownUps.com', '6265557265', '78934 Hillside Dr.', None, 'Pasadena', 'CA', '90003', 'USA', None, 'Young', 'Julie', 'Medium'),
 (29, 10140, 37, 100.0, 11, 7374.1, '7/24/2003 0:00', 'Shipped', 3, 7, 2003, 'Classic Cars', 214, 'S10_1949', 'Technics Stores Inc.', '6505556809', '9408 Furth Circle', None, 'Burlingame', 'CA', '94217', 'USA', None, 'Hirano', 'Juri', 'Large'),
 (30, 10150, 45, 100.0, 8, 10993.5, '9/19/2003 0:00', 'Shipped', 3, 9, 2003, 'Classic Cars', 214, 'S10_1949', 'Dragon Souveniers, Ltd.', '+65 221 7555', 'Bronz Sok., Bronz Apt. 3/6 Tesvikiye', None, 'Singapore', None, '79903', 'Singapore', 'Japan', 'Nativida